In [ ]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders - A warm-up challenge (~1h)

## Select features

🎯 Let's figure out the impact of `wait_time` and `delay_vs_expected` on very `good/bad reviews`

👉 Using our `orders` training_set, we will run two `multivariate logistic regressions`:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

 

In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Import your dataset:

In [2]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Select in a list which features you want to use:

⚠️ Make sure you are not creating data leakage (i.e. selecting features that are derived from the target)

💡 To figure out the impact of `wait_time` and `delay_vs_expected` we need to control for the impact of other features, include in your list all features that may be relevant

In [6]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt


from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

relevant_features = [
    'wait_time', 
    'delay_vs_expected', 
    'number_of_products', 
    'number_of_sellers', 
    'price', 
    'freight_value'
]


formula_one_star = 'dim_is_one_star ~ ' + ' + '.join(relevant_features)
formula_five_star = 'dim_is_five_star ~ ' + ' + '.join(relevant_features)
logit_one = smf.logit(formula=formula_one_star, data=orders).fit()
logit_five = smf.logit(formula=formula_five_star, data=orders).fit()

print(logit_one.summary())
print(logit_five.summary())

Optimization terminated successfully.
         Current function value: 0.274497
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.637411
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95865
Method:                           MLE   Df Model:                            6
Date:                Wed, 08 Nov 2023   Pseudo R-squ.:                  0.1419
Time:                        18:20:52   Log-Likelihood:                -26317.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

🕵🏻 Check the `multi-colinearity` of your features, using the `VIF index`.

* It shouldn't be too high (< 10 preferably) to ensure that we can trust the partial regression coefficents and their associated `p-values` 
* Do not forget to standardize your data ! 
    * A `VIF Analysis` is made by regressing a feature vs. the other features...
    * So you want to `remove the effect of scale` so that your features have an equal importance before running any linear regression!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardizing:

In [16]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler


features_for_vif = orders[[
    'wait_time', 
    'delay_vs_expected', 
    'number_of_products', 
    'number_of_sellers', 
    'price', 
    'freight_value'
]]

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_for_vif)
scaled_features_df = pd.DataFrame(scaled_features, columns=features_for_vif.columns)
scaled_features_df_with_constant = sm.add_constant(scaled_features_df)

vif_data = pd.DataFrame()
vif_data["feature"] = scaled_features_df_with_constant.columns
vif_data["VIF"] = [variance_inflation_factor(scaled_features_df_with_constant.values, i)
                   for i in range(scaled_features_df_with_constant.shape[1])]

print(vif_data)

              feature       VIF
0               const  1.000000
1           wait_time  2.104346
2   delay_vs_expected  2.022291
3  number_of_products  1.343611
4   number_of_sellers  1.093128
5               price  1.204782
6       freight_value  1.541776


👉 Run your VIF Analysis to analyze the potential multicolinearities:

In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import pandas as pd

def calculate_vif(dataframe, features):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(dataframe[features])
    scaled_features_df = pd.DataFrame(scaled_features, columns=features)
    scaled_features_df_with_constant = sm.add_constant(scaled_features_df)
    vif_data = pd.DataFrame()
    vif_data["feature"] = scaled_features_df_with_constant.columns
    vif_data["VIF"] = [variance_inflation_factor(scaled_features_df_with_constant.values, i)
                       for i in range(scaled_features_df_with_constant.shape[1])]
    
    return vif_data

features = [
    'wait_time', 
    'delay_vs_expected', 
    'number_of_products', 
    'number_of_sellers', 
    'price', 
    'freight_value'
]

vif_data = calculate_vif(orders, features)

for index, row in vif_data.iterrows():
    print(f"The VIF for {row['feature']} is {row['VIF']:.2f}, suggesting {'no' if row['VIF'] < 10 else 'significant'} multicollinearity.")



The VIF for const is 1.00, suggesting no multicollinearity.
The VIF for wait_time is 2.10, suggesting no multicollinearity.
The VIF for delay_vs_expected is 2.02, suggesting no multicollinearity.
The VIF for number_of_products is 1.34, suggesting no multicollinearity.
The VIF for number_of_sellers is 1.09, suggesting no multicollinearity.
The VIF for price is 1.20, suggesting no multicollinearity.
The VIF for freight_value is 1.54, suggesting no multicollinearity.


## Logistic Regressions

👉 Fit two `Logistic Regression` models:
- `logit_one` to predict `dim_is_one_star` 
- `logit_five` to predict `dim_is_five_star`.

`Logit 1️⃣`

In [18]:
formula_one_star = 'dim_is_one_star ~ wait_time + delay_vs_expected + number_of_products + number_of_sellers + price + freight_value'
logit_one = smf.logit(formula=formula_one_star, data=orders).fit()
print(logit_one.summary())

Optimization terminated successfully.
         Current function value: 0.274497
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95865
Method:                           MLE   Df Model:                            6
Date:                Wed, 08 Nov 2023   Pseudo R-squ.:                  0.1419
Time:                        18:31:57   Log-Likelihood:                -26317.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -5.3069      0.070    -75.412      0.000      -5.445      -5.169
wait_

`Logit 5️⃣`

In [17]:
# Define the formula for the logistic regression model to predict five-star reviews
formula_five_star = 'dim_is_five_star ~ wait_time + delay_vs_expected + number_of_products + number_of_sellers + price + freight_value'

# Fit the logistic regression model to predict five-star reviews
logit_five = smf.logit(formula=formula_five_star, data=orders).fit()

# Print the summary of the model for five-star reviews
print(logit_five.summary())

Optimization terminated successfully.
         Current function value: 0.637411
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95865
Method:                           MLE   Df Model:                            6
Date:                Wed, 08 Nov 2023   Pseudo R-squ.:                 0.05720
Time:                        18:31:37   Log-Likelihood:                -61110.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              2.4664      0.064     38.349      0.000       2.340       2.593
wait_

💡 It's time to analyse the results of these two logistic regressions:

- Interpret the partial coefficients in your own words.
- Check their statistical significances with `p-values`
- Do you notice any differences between `logit_one` and `logit_five` in terms of coefficient importances?

In [19]:
# Among the following sentences, store the ones that are true in the list below

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more more than one_star"

your_answer = []

# Add the true statements based on the analysis
if logit_five.params['delay_vs_expected'] < logit_one.params['delay_vs_expected']:
    your_answer.append(a)
if abs(logit_five.params['wait_time']) > abs(logit_one.params['wait_time']):
    your_answer.append(b)

print(your_answer)

['delay_vs_expected influences five_star ratings even more than one_star ratings']


🧪 __Test your code__

In [20]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/francoisgirard/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/francoisgirard/code/francoisgirard51/04-Decision-Science/04-Logistic-Regression/data-logit/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Explanations and advanced concepts </i> -</summary>


> _All other thing being equal, the `delay factor` tends to increase the chances of getting stripped of the 5-star even more so than it affect the chances of 1-star reviews. Probably because 1-stars are really targeting bad products themselves, not bad deliveries_
    
❗️ However, to be totally rigorous, we have to be **more careful when comparing coefficients from two different models**, because **they might not be based on similar populations**!
    We have 2 sub-populations here: (people who gave 1-stars; and people who gave 5-stars) and they may exhibit intrinsically different behavior patterns. It may well be that "happy-people" (who tends to give 5-stars easily) are less sensitive as "grumpy-people" (who shoot 1-stars like Lucky-Luke), when it comes to "delay", or "price"...

</details>


## Logistic vs. Linear ?

👉 Compare the coefficients obtained from:
- A `Logistic Regression` to explain `dim_is_five_star`
- A `Linear Regression` to explain `review_score` 

Make sure to use the same set of features for both regressions.  

⚠️ Check that both sets of coefficients  tell  "the same story".

> YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE

🏁 Congratulations! 

💾 Don't forget to commit and push your `logit.ipynb` notebook !